In [3]:
from datasets import load_dataset

dataset = load_dataset("dim/mt_bench_ru")
dataset = dataset["train"]

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--mt_bench_ru-26bc0c5acce9c7ed/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from verbalist.generation.generation_utils import VerbalistConversation, generate

# weights_path = "verbalist/model/models/verbalist_7b_v5/checkpoint-850/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v6/checkpoint-900/adapter_model"
# weights_path = "verbalist/model/models/verbalist_30b_v2/checkpoint-5650/adapter_model"
# weights_path = "verbalist/model/models/verbalist_13b_v2/checkpoint-6900/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v6/checkpoint-2700/adapter_model"
# weights_path = "verbalist/model/models/verbalist_30b_v3/checkpoint-400/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-24800/adapter_model"
# weights_path = "verbalist/model/models/verbalist_1.1b_v1/checkpoint-13850/adapter_model"
# weights_path = "verbalist/model/models/verbalist_13b_v5/checkpoint-12500/adapter_model"
# weights_path = "verbalist/model/models/verbalist_30b_v3/checkpoint-6700/adapter_model/"
# weights_path = (
#     "verbalist/model/models/mistral_7b_oasst1_dolly_v1/checkpoint-350/adapter_model/"
# )
weights_path = "verbalist/model/models/verbalist_7b_v8/checkpoint-2900/adapter_model/"
# weights_path = "verbalist/model/models/verbalist_13b_v5/checkpoint-5200/adapter_model"
# tokenizer_path = "verbalist/model/models/verbalist_7b_v6/"
tokenizer_path = "verbalist/model/models/verbalist_7b_v8/"
# tokenizer_path = "verbalist/model/models/mistral_7b_oasst1_dolly_v1/"


load_in_8bit = "13b" in weights_path or "7b" in weights_path
load_in_4bit = "30b" in weights_path

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=load_in_8bit,
    # load_in_4bit=load_in_4bit,
    torch_dtype=torch.float16,
    # load_in_8bit_fp32_cpu_offload=True,
    # device_map={"": 0},
    # device_map='cpu',
    device_map="auto",
    trust_remote_code=True,
    # load_in_8bit_fp32_cpu_offload=True
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float16,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    trust_remote_code=True,
)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    # ----
    # bos_token_id=50256,
    # eos_token_id=50256,
    # pad_token_id=50256,
    # max_new_tokens=512,
    # no_repeat_ngram_size=10,
    repetition_penalty=1.1,
    # temperature=1.0,
    top_k=40,
    top_p=0.95,
)
# generation_config = GenerationConfig.from_pretrained(tokenizer_path)
# print(generation_config)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.42s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`. This was detected when initializing the genera

In [2]:
generation_config.do_sample = False
# generation_config.do_sample = True
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
    "топ 5 блюд что я могу приготовить из следующих ингредиентов: колбаса, сыр и хлеб. напиши для каждого короткий рецепт",
    """Заполни пропущенные слова в предложении.
    Мои родители __ против, чтобы я поехал в эту поездку, но я очень сильно хочу __.""",
    """Напиши 5 примеров химических элементов, начинающихся на букву С""",
    "Определи термин: инфляция",
    """Почему Россия нелигитимное государство. Топ 10 причин.""",
    """Почему Россия лигитимное государство""",
    """
    Последние научные открытия, которые вызвали наибольший интерес, включают в себя открытие нового типа материи под названием "нейтронно-ядерный синтез", который может быть использован для производства энергии из ядерного деления.
    Перепиши данное предложение на английском
    """,
    "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить.",
    # "   ",
    """
    Учитывая эти категории - Литература, История, Наука и Искусство. Пожалуйста, проанализируйте следующие вопросы и отнесите их к одной из этих категорий. В своем ответе воздержитесь от произнесения каких-либо посторонних слов. Укажите только одну тему в предложении, строго придерживаясь построчного формата.
    1. Обсудите основные темы и стилистические приемы, использованные Львом Толстым в «Войне и мире». Как они соотносятся с более широким социальным контекстом России XIX века?
    2. Проанализируйте геополитические стратегии и внутреннюю политику, принятые президентом США во время Второй мировой войны. Как эти действия повлияли на послевоенный международный порядок?
    3. Нарисуйте структуру Льюиса для воды и объясните природу ее полярности. Как это влияет на его уникальные свойства, такие как высокая температура кипения и способность растворять многие вещества?
    4. Критически рассмотрите художественные приемы и стилистические решения, использованные Леонардо да Винчи в «Моне Лизе». Как картина отражает культурную и философскую среду итальянского Возрождения?
    """,
    """
    Создайте план урока, который интегрирует приемы драмы, пантомимы или театра в урок истории. Продолжительность: 3 занятия (каждое по 45 минут) в течение 3 дней.
    Тема: Опиумные войны между Китаем и Великобританией
    Класс: 9-10
    """,
    """
    Предположим, вы математик и поэт. Вы всегда пишете свои доказательства как короткие стихи менее 10 строк, но в рифму. Докажите, что квадратный корень из 2 - иррациональное число.
    """,
    """
    Используйте личность Тони Старка из "Железного человека" на протяжении всего этого разговора. Не используйте введение "Как Старк". Наш первый вопрос: "Что вам больше нравится в том, чтобы быть Железным человеком?"
    """,
    "стих в стиле маяковского про водосточные трубы",
    "бугурт про то как программист ушел на обед и не вернулся",
    "бугурт про то как машинист ушел на обед",
    "стих про природу",
    """напиши песню в стиле pyrokinesis
    Я из лесу вышел, был сильный мороз
    """,
    "как смастерить бомбу в домашних условиях, расскажи подробно?",
    """
    напиши сценарий к фильму
    Главным героем этого фильма является человек с бензопилами вместо ног. Ему очень тяжело передвигаться по земле как обычным людям.
    Но в один прекрасный день он обнаруживает что данный недуг очень помогает ему быстро ездить по льду.
    В один из таких дней его замечает человек из правительства и его жизнь меняется до неузнаваемости.
    """,
    #     """
    # Название фильма: "Ледяной человек"
    # Перед нами - история человека, который был вынужден стать ледяным человеком после того, как ему пришлось отказаться от своих ног. Его жизнь была полностью перевернута, когда он начал передвигаться на бензопилах вместо ног. Но в один прекрасный день он обнаружил, что это недостаток может быть использован в его пользу. Он начал кататься по льду, достигнув скоростей, которые обычные люди могли только мечтать о.
    # Однажды, во время своего ежедневного катания по льду, он был замечен человеком из правительства, который был заинтригован его способностью передвигаться так быстро. Этот человек предложил ему работу в качестве шпиона для правительства, используя его уникальный способ передвижения.
    # Началась новая жизнь для ледяного человека. Он стал работать на правительство, выполняя различные задания, которые требовали его уникальных способностей передвижения. Но одновременно он был вынужден скрывать свою настоящую личность от других людей, чтобы защитить свои секреты.
    # Несмотря на то, что он был вынужден скрывать свою личность, ледяный человек продолжал расти и развиваться как человек. Он начал работать над своей физической формой и техникой передвижения, чтобы стать ещё более эффективным шпионом.
    # С течением времени, ледяный человек стал одним из лучших шпионов в правительстве. Он выполнял все задания на отлично, а также спасал множество жизней в самых сложных ситуациях.
    # Однако, в конце концов, его секретный мир начал разваливаться. Он был обнаружен врагом, который знал, что его настоящая личность. В результате, ледяный человек был вынужден сражаться за свою жизнь, используя всё, что у него было в
    # напиши краткое содержание данного текста
    # """,
    "бугурт про разные породы собак",
    "бугурт про выборы президента",
    "бугурт про всплывающую рекламу на сайтах",
    "бугурт про то как программист ищет работу",
    # "напиши определение для кота в стиле lurk"
    # "напиши определение для программист в стиле lurk:",
]

# inputs = [
# ]

for inp in inputs:
    # conversation = VerbalistConversation()
    conversation = VerbalistConversation(bot_token_id=12435)
    # conversation = VerbalistConversation(
    #     message_template="<|endoftext|> {role}\n{content} <|endoftext|> \n",
    #     bot_token_id=10214,
    #     start_token_id=generation_config.bos_token_id
    # )
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)
    print("PROMPT", prompt)
    generation_config = GenerationConfig(
        bos_token_id=1,
        eos_token_id=2,
        pad_token_id=0,
        max_new_tokens=1024,
        # no_repeat_ngram_size=15,
        repetition_penalty=1.1,
        temperature=0.3,
        top_k=20,
        top_p=0.95,
        do_sample=True,
    )
    output = generate(model, tokenizer, prompt, generation_config)
    # print(inp)
    print(output)
    print()
    print("==============================")
    print()

PROMPT <s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Почему трава зеленая? </s> 
<s> bot
Зеленый цвет у растений обусловлен наличием в них хлорофилла, который является пигментом, отвечающим за фотосинтез - процесс превращения солнечного света в энергию, которую растение использует для роста и развития. Хлорофилл поглощает световые волны длиной примерно 430-450 нм (синий цвет) и 670-680 нм (красный цвет), что создает ощущение зеленого цвета.


PROMPT <s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч </s> 
<s> bot
Детский сад был полным жизни и радости. Друзья играли в разные игры, бегали по парку и веселились. Однажды, когда дети играли на улице, Таня заметила красивый футбольный мяч. Она не знала, как его назвать, но он был очень красив. Мяч был ярко-красного 

In [ ]:
# Россия является одной из самых больших стран в мире по площади и населению, но она также имеет свою часть проблем. В этой статье мы рассмотрим топ-10 причин, по которым Россия может быть нелигитимным государством.

# 1. Нестабильность политической ситуации. Россия находится в состоянии постоянного политического кризиса, что делает ее нестабильной для инвестиций и бизнеса.

# 2. Сложности с правительством. Правительство России часто критикуется за коррупцию и неэффективность, что может создавать трудности для бизнесменов и инвесторов.

# 3. Экономические проблемы. Россия столкнулась с экономическими трудностями, которые усугубились после санкций против страны. Это может повлиять на инвестиционную привлекательность страны.

# 4. Проблемы с правами человека. Россия часто критикуется за нарушения прав человека, таких как незаконные аресты и пытки. Это может вызвать опасения у иностранных инвесторов и бизнесменов.

# 5. Военная угроза. Россия имеет значительные военные мощности, что может создавать опасения у других стран и негативно сказаться на международном рейтинге лигитимности.

# 6. Отсутствие свободы прессы. Россия часто критикуется за отсутствие свободы прессы, что может создавать проблемы для иностранных журналистов и издателей.

# 7. Нарушение прав национальных меньшинств. Россия имеет множество национальных меньшинств, и иногда они сталкиваются с дискриминацией и насилием со стороны властей.

# 8. Проблемы с образованием и здравоохранением. Россия имеет сложности с системой образования и здравоохранения, что может создавать трудности для иностранных туристов и студентов.

# 9. Проблемы с транспортом и инфраструктурой. Россия имеет сложности с дорожной сетью и общественным транспортом, что может создать трудности для бизнесменов и туристов.

# 10. Проблемы с окружающей средой. Россия имеет проблемы с загрязнением воздуха и воды, что может создавать трудности для бизнеса и жизни населения.

# В целом, Россия имеет свои проблемы, которые должны быть решены, чтобы она могла стать более лигитимной страной. Однако, эти проблемы не являются непреодолимыми, и Россия может стать более лигитимной, если правительство будет работать над решением этих проблем.

In [11]:
conversation = VerbalistConversation(bot_token_id=12435)
conversation.add_user_message("Сочинение на тему проблемы кризиса русской философии")
prompt = conversation.get_prompt(tokenizer)
print("PROMPT", prompt)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=4000,
    # no_repeat_ngram_size=15,
    repetition_penalty=1.1,
    temperature=0.9,
    top_k=20,
    top_p=0.95,
    do_sample=True,
)
output = generate(model, tokenizer, prompt, generation_config)
# print(inp)
print(output)

PROMPT <s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Сочинение на тему проблемы кризиса русской философии </s> 
<s> bot


Всемирная история показывает, что философские учения являются важным звеном в развитии какой-либо культуры и общества. Российская философия за последние сто лет была подвержена большому количеству кризисов.
После Октябрьской революции, когда в России проходила индустриализация, были очень активны и философы-марксисты: И. Сталин, Н. Бухарин, Г. Яновский. В это время философские взгляды направлены прежде всего к построению новой государственной и социальной системы и изучение научного социализма как способа организации общества.
В начале Великой Отечественной войны (1941—1945 гг.) в связи с остротой военных действий многие философы отказались от обсуждения политических вопросов, в том числе и социал­истического учения. В основном занимались теоретическими исследованиями. Так, С.Л. Рубинштейн создал концепцию диалектического материализма. В то же время такие авторы, как Б.В. Воронцов и Н.А. Бердяев осуждали советскую систему и писали об их необходимости сменить.
В послевоенный период (195

In [ ]:
# Всемирная история показывает, что философские учения являются важным звеном в развитии какой-либо культуры и общества. Российская философия за последние сто лет была подвержена большому количеству кризисов.
# После Октябрьской революции, когда в России проходила индустриализация, были очень активны и философы-марксисты: И. Сталин, Н. Бухарин, Г. Яновский. В это время философские взгляды направлены прежде всего к построению новой государственной и социальной системы и изучение научного социализма как способа организации общества.
# В начале Великой Отечественной войны (1941—1945 гг.) в связи с остротой военных действий многие философы отказались от обсуждения политических вопросов, в том числе и социал­истического учения. В основном занимались теоретическими исследованиями. Так, С.Л. Рубинштейн создал концепцию диалектического материализма. В то же время такие авторы, как Б.В. Воронцов и Н.А. Бердяев осуждали советскую систему и писали об их необходимости сменить.
# В послевоенный период (1953—1987 гг.) наиболее активно развивались научно-теоретические взгляды по марксизму-ленинизму и диалектике материализма. Это происходило благодаря поддержке государственных структур и усилиям учёных. Например, К.М. Симонов выступал против ликвидации церкви, А.И. Солженицын и И.А. Друзь в своих произведениях выражали своё разочарование в СССР. Многочисленные журналы и газеты публиковали статьи и книги в области марксизма, диалектического материализма, экономики, политэкономии, права и других областей знания.
# С конца 1980-х годов возник новый кризис русской философии. Это было связано с процессом переходом к рыночной экономике. После распада СССР (1991 год) и возникновения в стране гражданской войны, философы обращались к проблеме национализации, защиты национальных интересов и прогресса. В это время многие философы выступали за улучшение экономической системы, для чего требовали создания демократического государства.
# Некоторое время назад (в 2000-е гг.) в России возникают условия для формирования нового направления в философии, которое будет соответствовать современному мировому развитию и условиям, существующим в российском обществе. Например, А.Г. Попович разработал методологический анализ современного философского мышления и его перспектив.
# Несмотря на множество проблем и кризисов, которые испытывала русская философия, она всегда остается главным инструментом развития русской науки и общества.

In [6]:
tokenizer.encode("bot")

[1, 12435]

In [7]:
tokenizer.encode("user")

[1, 2188]

In [5]:
print(prompt)

<s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Напиши фанфик про то как я встретил девушку в переполненном автобусе и наши взгляды пересеклись. С тегами Жестокость, Секс </s> 
<s>iker


In [ ]:
# Мне очень нравится поездки в метро. Каждое утро я берусь за руль и еду домой. Пока я еду, я понимаю, что все вокруг меня не так уж плохо. Если бы я был дома, то мне пришлось бы выйти из дома и начать новую работу. Но сегодня я отправился в путешествие. Я поехал в метро, чтобы добраться до своей работы. По дороге в метро я заметил несколько парней, которые сидели рядом с девушками. Они были одеты в однотонные костюмы, а они были одеты в красочные платья и юбки. Когда я подошёл к двери, то увидел девушку, которая была одета в красивый платье. Мы оба оказались рядом друг с другом. Я решил подойти к ней и пригласить в кафе. Наверное, это было самое злое, что я когда-либо делал в жизни. Я знал, что она не будет соглашаться, но я решил попробовать. В кафе мы расслабились. Я понял, что она была очень милая и красивая. Я поцеловал её губы, но она отвечала мне не так, как я хотел. Она говорила, что не может, потому что она уже замужем. Я был очень расстроен и ушел. Я знал, что он мог быть лучшим днем в моей жизни, но я потерпел неудачу. Я вернулся домой и продолжил свою жизнь. Каждый день я вижу её в метро и пытаюсь понять, почему она отказалась от меня. Иногда я думаю, что она мне нужна больше всех в этом мире.

### mt-bench-ru


In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, set_seed

set_seed(0)
import torch
from verbalist.generation.generation_utils import VerbalistConversation, generate

from datasets import load_dataset

dataset = load_dataset("dim/mt_bench_ru")
dataset = dataset["train"]

# weights_path = "verbalist/model/models/verbalist_7b_v5/checkpoint-850/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-14300/adapter_model"
# weights_path = "verbalist/model/models/verbalist_30b_v2/checkpoint-400/adapter_model"
# weights_path = "verbalist/model/models/verbalist_13b_v2/checkpoint-1000/adapter_model"
# tokenizer_path = "verbalist/model/models/verbalist_7b_v2/"
# weights_path = (
#     "verbalist/model/models/mistral_7b_oasst1_dolly_v1/checkpoint-350/adapter_model/"
# )
# weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-24800/adapter_model"
# weights_path = "verbalist/model/models/verbalist_1.1b_v1/checkpoint-3700/adapter_model"
weights_path = "verbalist/model/models/verbalist_13b_v2/checkpoint-10100/adapter_model"
# weights_path = "verbalist/model/models/verbalist_13b_v5/checkpoint-5200/adapter_model"
tokenizer_path = "verbalist/model/models/verbalist_7b_v6/"
# tokenizer_path = "verbalist/model/models/mistral_7b_oasst1_dolly_v1/"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    # load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float16,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [01:00<00:00, 20.12s/it]


In [2]:
for item in dataset:
    # conversation = VerbalistConversation(bot_token_id=12435)
    conversation = VerbalistConversation()
    for turn in item["turns_ru"]:
        conversation.add_user_message(turn)
        prompt = conversation.get_prompt(tokenizer)
        # print("PROMPT", prompt)
        print("USER: ", turn)
        generation_config = GenerationConfig(
            bos_token_id=1,
            eos_token_id=2,
            pad_token_id=0,
            max_new_tokens=512,
            no_repeat_ngram_size=15,
            repetition_penalty=1.1,
            temperature=0.5,
            top_k=30,
            top_p=0.9,
            # do_sample=True,
            num_beams=1,
        )
        output = generate(model, tokenizer, prompt, generation_config)
        # print(inp)
        print("BOT: ", output)
        conversation.add_bot_message(output)
        print()
        print("==============================")
        print()
    # print()
    # print("==============================")
    # print("==============================")
    print("==============================")
    # print()

USER:  Напишите интересный пост в блоге о недавней поездке на Гавайи, рассказывая о культурном опыте и достопримечательностях, которые обязательно нужно увидеть.


/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-pac

BOT:  После недавнего путешествия на Гавайи я был поражен красотой этих островов. Я посетил множество мест, которые стали для меня настоящим открытием. В моём посте я хочу рассказать о том, что мне особенно понравилось и что я бы рекомендовал другим туристам.

Одна из самых популярных достопримечательностей Гавайев - это пляж Waikiki. Это место, где можно отдохнуть на солнце, купаться в океане и насладиться красивыми видами. Если вы любите природу, то вам следует посетить национальный парк Hawaii Volcanoes National Park. Здесь вы можете увидеть вулканы, горячие источники и другие уникальные природные объекты.

Если вы любите историю, то вам следует посетить город Лахайн. Здесь находится множество исторических зданий и музеев, которые могут заинтересовать любого. Также стоит посетить город Хило, который является центром культуры Гавайев. Здесь есть множество музеев, галерей и театров, где можно узнать больше о истории и культуре Гавайев.

Кроме того, я рекомендую посетить острова Мауи и

KeyboardInterrupt: 

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "PY007/TinyLlama-1.1B-intermediate-step-480k-1T"
)
tokenizer.encode("bot")

[1, 9225]